In [1]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D,GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import cv2
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
input_shape = (105, 105, 3)
left_input = Input(input_shape)
right_input = Input(input_shape)

In [3]:
def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
#//TODO: figure out how to initialize layer biases in keras.
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=rng.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

In [4]:
# 构建不带分类器的预训练模型
base_model = InceptionV3(weights='imagenet', include_top=False)
# 添加全局平均池化层
x = base_model.output
x = GlobalAveragePooling2D()(x)

# 添加一个全连接层
embedding = Dense(1024, activation='relu',kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init)(x)

# 构建我们需要训练的完整模型
convnet = Model(inputs=base_model.input, outputs=embedding)

In [5]:
convnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [6]:
#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1024)         23900960    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1024)         0           model_1[1][0]                    
          

In [7]:
# 锁住所有 InceptionV3 的卷积层
for layer in base_model.layers:
    layer.trainable = False

In [8]:
siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1024)         23900960    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1024)         0           model_1[1][0]                    
          

In [9]:
optimizer = Adam(0.00006)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

In [10]:
PATH = "./Bongard/BP_61" #CHANGE THIS - path where the pickled data is stored

with open(os.path.join(PATH, "train.pickle"), "rb") as f:
    (X,c) = pickle.load(f)

with open(os.path.join(PATH, "val.pickle"), "rb") as f:
    (Xval,cval) = pickle.load(f)
    
print("training alphabets")
print(c.keys())
print("validation alphabets:")
print(cval.keys())

training alphabets
dict_keys(['left', 'right'])
validation alphabets:
dict_keys(['left', 'right'])


In [11]:
class Siamese_Loader:
    """For loading batches and testing tasks to a siamese net"""
    def __init__(self, path, data_subsets = ["train", "val"]):
        self.data = {}
        self.categories = {}
        self.info = {}
        
        for name in data_subsets:
            file_path = os.path.join(path, name + ".pickle")
            print("loading data from {}".format(file_path))
            with open(file_path,"rb") as f:
                (X,c) = pickle.load(f)
                self.data[name] = X
                self.categories[name] = c

    def get_batch(self,batch_size,s="train"):
        """Create batch of n pairs, half same class, half different class"""
        X=self.data[s]
        n_classes, n_examples, w, h = X.shape

        #randomly sample several classes to use in the batch
        categories = rng.choice(n_classes,size=(batch_size,),replace=False)
        #initialize 2 empty arrays for the input image batch
        #pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
        pairs=[np.zeros((batch_size, h, w,3)) for i in range(2)]
        #initialize vector for the targets, and make one half of it '1's, so 2nd half of batch has same class
        targets=np.zeros((batch_size,))
        targets[batch_size//2:] = 1
        for i in range(batch_size):
            category = categories[i]
            idx_1 = rng.randint(0, n_examples)
            #pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
            
            pairs[0][i,:,:,:] = cv2.cvtColor(cv2.resize(X[category, idx_1], (w, h)), cv2.COLOR_GRAY2RGB)
            idx_2 = rng.randint(0, n_examples)
            #pick images of same class for 1st half, different for 2nd
            if i >= batch_size // 2:
                category_2 = category  
            else: 
                #add a random number to the category modulo n classes to ensure 2nd image has
                # ..different category
                category_2 = (category + rng.randint(1,n_classes)) % n_classes
            #pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
            pairs[1][i,:,:,:]=cv2.cvtColor(cv2.resize(X[category_2,idx_2], (w, h)), cv2.COLOR_GRAY2RGB)
        return pairs, targets
    
    def generate(self, batch_size, s="train"):
        """a generator for batches, so model.fit_generator can be used. """
        while True:
            pairs, targets = self.get_batch(batch_size,s)
            yield (pairs, targets)   
            
    def test_bongard(self,model,N,k,s='val',s1='train',verbose=0):
        
        X_train=self.data[s1]
        X_val=self.data[s]
        
        n_classes, n_examples, w, h = X_train.shape
        m = n_classes*n_examples
        X_train=X_train.reshape(m, w, h,1)
        X_train = [cv2.cvtColor(cv2.resize(i, (w, h)), cv2.COLOR_GRAY2RGB) for i in X_train]
        X_train=np.array(X_train)

        
        n_classes_val, n_examples_val, w, h = X_val.shape
        m_val = n_classes_val * n_examples_val
        X_val=X_val.reshape(m_val, w, h,1)
        X_val = [cv2.cvtColor(cv2.resize(i, (w, h)), cv2.COLOR_GRAY2RGB) for i in X_val]
        X_val=np.array(X_val)
        
        
        n_correct = 0
        
        for i in range(k):
            
            test_image = np.asarray([X_val[i,:,:,:]]*m).reshape(m, w, h,3)
            support_set =  X_train
            if i < k/2:
                targets=0
            else:
                targets=1 
            
            pairs = [test_image,support_set]
            
            probs = model.predict(pairs)
            if np.argmax(probs)<6:
                test_result=0
            else:
                test_result=1
            if test_result== targets:
                n_correct+=1
                
        percent_correct = (100.0*n_correct / k)
        if verbose:
            print("Got an average of {}% {} way one-shot learning accuracy".format(percent_correct,N))
        return percent_correct
            
    def train(self, model, epochs, verbosity):
        model.fit_generator(self.generate(batch_size),
                            
                             )
    
    
#Instantiate the class
loader = Siamese_Loader(PATH)

loading data from ./Bongard/BP_61/train.pickle
loading data from ./Bongard/BP_61/val.pickle


In [ ]:
#Training loop
print("!")
evaluate_every = 1 # interval for evaluating on one-shot tasks
loss_every=50 # interval for printing loss (iterations)
batch_size = 2
n_iter = 200
N_way = 2 # how many classes for testing one-shot tasks>
n_val = 500 #how mahy one-shot tasks to validate on?
best = 0.9
weights_path = os.path.join(PATH, "weights")
print("training")
for i in range(1, n_iter):
    (inputs,targets)=loader.get_batch(batch_size)
    loss=siamese_net.train_on_batch(inputs,targets)
    print(loss)
    if i > n_iter/2:
        print("evaluating")
        val_acc = loader.test_bongard(siamese_net,N_way,n_val,s='val',s1='train',verbose=True)
        if val_acc >= best:
            print("saving")
            siamese_net.save(weights_path)
            best=val_acc

    if i % loss_every == 0:
        print("iteration {}, training loss: {:.2f},".format(i,loss))

!
training
0.94372183
0.9085449
0.9146389
0.9373926
0.90785265
0.9036267
0.96286047
0.91288817
0.94283926
0.92144364
1.0040646
0.92958236
0.9320109
0.8148593
0.9107969
0.9403581
0.93550503
0.7865073
0.9455202
0.942155
0.68512124
0.6863448
0.9465091
0.8749959
0.9136363
1.0057139
0.9020987
0.7239487
0.9000901
0.7176686
0.98545086
1.1920632
0.95419025
0.7594095
0.95818603
1.0128059
0.73720974
1.0031104
0.7193648
0.9244555
0.88559616
0.8975525
0.9366995
0.91945404
0.9840623
0.8859349
0.8240072
0.9097442
0.91217923
1.3107026
iteration 50, training loss: 1.31,
0.92500126
1.0257459
0.91392434
0.71706235
0.89691544
0.9884621
0.8783338
0.8330555
0.92609185
1.099679
0.90841275
1.0517755
0.96027076
0.9071127
0.7917215
0.6940529
0.5351683
1.1634722
1.1764176
1.5203221
1.0488359
0.7716947
0.96089864
0.8453461
0.9476061
0.9804039
0.94210136
1.2790179
0.8974834
0.7178687
1.0257787
0.9543254
0.88191926
0.92524457
0.88303715
1.0627792
1.1520734
0.9735534
0.82033825
0.67506766
0.95550543
1.003597
0.9820